# 데이터 인코딩

## 레이블 인코딩(label encoding)

- 카테고리 피처를 코드형 숫자 값으로 변환
- 예: 상품 데이터의 상품 구분이 TV, 냉장고, 전자레인지, 컴퓨터로 되어 있다면 TV = 1, 냉장고 = 2, 전자레인지 = 3, 컴퓨터 = 4와 같은 숫자형 값으로 변환
- 레이블 인코딩을 사용할 경우 특정 머신러닝 알고리즘은 1보다 2가 더 큰 값이므로 이에 가중치를 더 부여하거나 더 중요하게 인식함(냉장고보다 컴퓨터를 더 중요하게 인식)
- 따라서 레이블 인코딩은 선형회귀와 같은 머신러닝 알고리즘에는 적용하지 않아야 함
- 트리 계열의 머신러닝 알고리즘은 숫자의 이러한 특성을 반영하지 않으므로 레이블 인코딩도 문제 없음

In [1]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 변환값: ', labels)
print('인코딩 클래스: ', encoder.classes_)

인코딩 변환값:  [0 1 4 5 3 3 2 2]
인코딩 클래스:  ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


- TV = 0, 냉장고 = 1, 믹서 = 2, 선풍기 = 3, 전자레인지 = 4, 컴퓨터 = 5

In [2]:
# 인코딩된 값을 다시 디코딩

print('디코딩 원본값: ', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

디코딩 원본값:  ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


## 원핫인코딩(one hot encoding)

- 레이블 인코딩의 문제점 해결
- 피처 값의 유형에 따라 새로운 피처를 추가해 고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시하는 방식
- 행 형태로 돼 있는 피처의 고유 값을 열 형태로 차원을 변환한 뒤, 고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시함
- 라벨인코딩과 달리 입력값으로 2차원 데이터가 필요함
- onehotencoder를 이용해 변환한 값이 희소 행렬(sparse matrix)형태이므로 이를 다시 toarray()메서드를 이용해 밀집행렬(dense matrix)로 변환해야 함

In [4]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# 2차원 ndarray로 변환
items = np.array(items).reshape(-1, 1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬이므로 toarray()를 이용해 밀집 행렬로 변환
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원: ', oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원:  (8, 6)


In [5]:
# 원-핫 인코딩을 더 쉽게 지원하는 API - get_dummies()
## 사이킷런의 OneHotEncoder와 다르게 문자열 카테고리 값을 숫자형으로 변환할 필요 없이 바로 변환

import pandas as pd

df = pd.DataFrame({'item': ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 학습 데이터와 테스트 데이터의 스케일링 변환 시 유의점

- scaler 객체를 이용해 학습 데이터 세트로 fit()과 transform()을 적용하면 테스트 데이터 세트로는 다시 fit()을 수행하지 않고 학습 데이터 세트로 fit()을 수행한 결과를 이용해 transform() 변환을 적용해야 함

In [6]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0부터 10까지, 테스트 데이터는 0부터 5까지 값을 가지는 데이터 세트로 생성
# scaler 클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1, 1)로 차원 변경

train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

scaler = MinMaxScaler()

# 학습 데이터에 minmaxscaler객체의 fit()을 적용하면 최솟값 0, 최대값 10이 설정되며 1/10 scale이 적용됨
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)

print('원본 train_array 데이터: ', np.round(train_array.reshape(-1), 2))
print('scaled된 train_array 데이터: ', np.round(train_scaled.reshape(-1), 2))

원본 train_array 데이터:  [ 0  1  2  3  4  5  6  7  8  9 10]
scaled된 train_array 데이터:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [7]:
# 테스트 데이터 세트를 변환하는데 fit()을 호출해 스케일링 기준 정보를 다시 적용한 뒤 transform()을 수행
## 잘못된 방법

# 테스트 데이터에 fit()을 적용하면 최솟값 0, 최댓값 5로 설정됨
scaler.fit(test_array)

# 1/5 scaler로 test_array 데이터 변환.
test_scaled = scaler.transform(test_array)

print('원본 test_array 데이터: ', np.round(test_array.reshape(-1), 2))
print('scale된 test_array 데이터: ', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터:  [0 1 2 3 4 5]
scale된 test_array 데이터:  [0.  0.2 0.4 0.6 0.8 1. ]


- 출력 결과를 확인하면 학습 데이터와 테스트 데이터의 스케일링이 맞지 않음을 알 수 있음
- 이렇게 되면 학습 데이터와 테스트 데이터의 서로 다른 원본값이 동일한 값으로 변환되는 결과를 초래함
- 머신러닝 모델은 학습 데이터를 기반으로 학습되기 때문에 반드시 테스트 데이터는 학습 데이터의 스케일링 기준에 따라야 하며, 테스트 데이터의 1 값은 학습 데이터와 동일하게 0.1값으로 변환돼야 함
- 따라서 테스트 데이터에 다시 fit()을 적용해서는 안되며 학습 데이터로 이미 fit()이 적용된 scaler 객체를 이용해 transform()으로 변환해야 함

In [8]:
# 옳은 방법
## 테스트 데이터에 fit()을 호출하지 않고 학습 데이터로 fit()을 수행한 MinMaxScaler 객체의 transform()을 이용해 데이터 변환

scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터: ', np.round(train_array.reshape(-1), 2))
print('scale된 train_array 데이터: ', np.round(train_scaled.reshape(-1), 2))

# test_array에 scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform()만으로 변환해야 함
test_scaled = scaler.transform(test_array)
print('\n원본 test_array 데이터: ', np.round(test_array.reshape(-1), 2))
print('scale 된 test_array 데이터: ', np.round(test_scaled.reshape(-1), 2))

원본 train_array 데이터:  [ 0  1  2  3  4  5  6  7  8  9 10]
scale된 train_array 데이터:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

원본 test_array 데이터:  [0 1 2 3 4 5]
scale 된 test_array 데이터:  [0.  0.1 0.2 0.3 0.4 0.5]


#### 이런 주의사항이 있으므로 학습과 테스트 데이터 세트로 분리하기 전에 먼저 전체 데이터 세트에 스케일링을 적용한 뒤 학습과 테스트 데이터 세트로 분리하는 것이 더 바람직함

- 이 유의사항은 사이킷런 기반의 PCA와 같은 차원 축소 변환이나 텍스트의 피처 벡터화 변환 작업 시에도 동일하게 적용됨